In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Importing Libraries

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
import math
import numpy as np
import pandas as pd
import scipy.stats as ss
import cufflinks as cf
from collections import Counter

import re
import nltk
from nltk.corpus import stopwords
from textblob import TextBlob
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm

import keras
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, SpatialDropout1D, Bidirectional
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import roc_auc_score
from keras.preprocessing.text import Tokenizer


# Loading Data

In [ ]:
data = pd.read_csv('../input/Tweets.csv')


# Exploring Data


Pandas head() method is used to return top 100 rows of a data frame or series.

In [ ]:
data.head(100)


This method prints information about a DataFrame including the index dtype and column dtypes, non-null values and memory usage.


In [ ]:
data.info()


looping on the data columns to get the columns names and its count

In [ ]:
column_names = []
column_counts = []
for column in data:
    column_names.append(column)
    column_counts.append(data[column].count())

visualize the data columns names and counts

In [ ]:
#visualize columns count
plt.subplots(figsize=(25,20))
sns.barplot(x=column_names, y=column_counts)

found out that airline sentiment gold and negative reason gold and tweet coord have too many null data to be important

## data correlation

In [ ]:
#visualize columns names
list(data.columns)


* ## airline_sentiment distribution

creating probability distributions for some columns of my data frame by distplot from seaborn library sns.distplot().

In [ ]:
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

data['airline_sentiment'].iplot(
    kind='hist',
    bins=50,
    xTitle='polarity',
    linecolor='black',
    yTitle='count',
    title='airline_sentiment Distribution')

In [ ]:
data['text'].iplot(
    kind='hist',
    bins=50,
    xTitle='polarity',
    linecolor='black',
    yTitle='count',
    title='text Distribution')

In [ ]:
data = data.drop_duplicates()

remove duplicated rows to clean the data

## apply histogram to see the distribution of data

A histogram is used to summarize discrete or continuous data. In other words, it provides a visual interpretation of numerical data by showing the number of data points that fall within a specified range of values (called “bins”). It is similar to a vertical bar graph

In [ ]:
data['text_length'] = data['text'].apply(len)
hist = data.hist(bins=4)

## calc mean of retweet count for each category to decide if this feature is important

In [ ]:
dp=data[ data['airline_sentiment'] == 'positive']
dg=data[ data['airline_sentiment'] == 'negative']
dn=data[ data['airline_sentiment'] == 'neutral']
positive_retweet_mean=dp['retweet_count'].mean()
negative_retweet_mean=dg['retweet_count'].mean()
neutral_retweet_mean=dn['retweet_count'].mean()
print("positive_retweet_mean", positive_retweet_mean)
print("negative_retweet_mean", negative_retweet_mean)
print("neutral_retweet_mean",neutral_retweet_mean)

Compute pairwise correlation of columns, excluding NA/null values.



In [ ]:
data.corr()


In [ ]:
def correlation_ratio(categories, measurements):
    fcat, _ = pd.factorize(categories)
    cat_num = np.max(fcat)+1
    y_avg_array = np.zeros(cat_num)
    n_array = np.zeros(cat_num)
    for i in range(0,cat_num):
        cat_measures = measurements[np.argwhere(fcat == i).flatten()]
        n_array[i] = len(cat_measures)
        y_avg_array[i] = np.average(cat_measures)
    y_total_avg = np.sum(np.multiply(y_avg_array,n_array))/np.sum(n_array)
    numerator = np.sum(np.multiply(n_array,np.power(np.subtract(y_avg_array,y_total_avg),2)))
    denominator = np.sum(np.power(np.subtract(measurements,y_total_avg),2))
    if numerator == 0:
        eta = 0.0
    else:
        eta = np.sqrt(numerator/denominator)
    return eta

## Data Analysis

select rows in my data frame contains negative the choose the rows from the selected columns in the second line meets up to the selected rows, group it bu airline and sorted by negative reasons

In [ ]:
#rank for the bad airline
negative_tweets = data[data['airline_sentiment'].str.contains("negative")]
bad_airline = negative_tweets[['airline','airline_sentiment_confidence','negativereason']]
bad_airline_count = bad_airline.groupby('airline', as_index=False).count()
bad_airline_count.sort_values('negativereason', ascending=False)

select rows in my data frame contains positive the choose the rows from the selected columns in the second line meets up to the selected rows, group it bu airline and sorted by negative reasons

In [ ]:
#rank for the good airline
positive_tweets = data[data['airline_sentiment'].str.contains("positive")]
good_airline = positive_tweets[['airline','airline_sentiment_confidence']]
good_airline_count = good_airline.groupby('airline', as_index=False).count()
good_airline_count.sort_values('airline_sentiment_confidence', ascending=False)


visualize reasons based on the negative tweets of the selected columns

In [ ]:
reason = negative_tweets[['airline','negativereason']]
bad_flight_reason_count = reason.groupby('negativereason', as_index=False).count()
bad_flight_reason_count.sort_values('negativereason', ascending=False)

## Create df with the important columns

after visualizations and analysis, i selected the columns that seemed to be most important to create new data frame

In [ ]:
data_new = pd.read_csv('../input/Tweets.csv')
df = pd.DataFrame([])
df['airline_sentiment'] = data_new['airline_sentiment']
df['text'] = data_new['text']
df['negativereason'] = data_new['negativereason']

based on correlation: airline_sentiment,text and negativereason correlates more with the target

Pandas head() method is used to return top 5 -by default- rows of a data frame or series.

In [ ]:
df.head()

# Preprocessing Text Data

## Utils

We create a normalizer class that is responsible for doing preprocessing and cleaning functions like removing punctuations, mentions, urls, emails, stopwords

In [ ]:
class Normalizer:
    def __init__(self):
        self.stop_words = stopwords.words('english')
    
    def lower(self, text):
        return text.lower()
    
    def remove_punctuations(self, text):
        return re.sub('[^\w\s]','', text)
    
    def remove_mentions(self, text):
        return re.sub('@[a-zA-Z0-9-._]+', '', text)
    
    def remove_url(self, text):
        url_ptrn = r'''(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|
            www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|
            \(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|
            (\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»""'']))'''
        return re.sub(url_ptrn, '', text)
    
    def remove_email(self, text):
        return re.sub(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+',
                             '', text)
    def remove_stop_words(self, text):
        return " ".join(x for x in text.split() if x not in self.stop_words)
    
    def correct_spelling(self, text):
        return str(TextBlob(text).correct())
    
    def tokenize(self, text):
        return word_tokenize(text)
    
    def normalize(self, text):
        text = self.lower(text)
        text = self.remove_url(text)
        text = self.remove_email(text)
        text = self.remove_mentions(text)
# #         text = self.correct_spelling(text)
#         text = self.remove_stop_words(text)
        text = self.remove_punctuations(text)
        text = text.strip()
        return text

We initialize the Normalizer

In [ ]:
normalizer = Normalizer()


We apply all the selected normalizer operations on the text column in our dataframe

In [ ]:
df.text = df.text.apply(normalizer.normalize)


### word cloud

we create a Word Cloud to visualize the most occuring words in our data

In [ ]:
def word_cloud(text, Stop_words=None,path=None,height=3000, width=3000):
    wc = WordCloud(background_color="white", max_words=2000,
                contour_width=3, contour_color='steelblue', stopwords=Stop_words,height=3000,width=3000).generate(text)
    wc.to_file(path)
    plt.imshow(wc, interpolation='bilinear', shape=(width, height ))
    plt.axis("off")
    plt.figure()

all_text = ''
for row in df.text:
    all_text += ' ' + str(row)
word_cloud(all_text, path='Cloud.png')


# Machine Learning Model

## preprocessing for ML model

We create a tf-idf vectorizer to represent our data in format that would be acceptable for the ML models.

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1,1))
X = vectorizer.fit_transform(df.text)
y = df.airline_sentiment

### split the data into train and test

we use stratify to equally split the three classes into train and test groups

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)


## create ml model

### MNB Classifier

Naive Bayes classifier for multinomial models

The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf works well.

In [ ]:
clf_mnb = MultinomialNB().fit(X_train, y_train)



### SVM Classifier

Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.
LinearSVC is capable of performing multi-class classification on a dataset.

In [ ]:
clf_svm = svm.LinearSVC(penalty='l2',C=1).fit(X_train, y_train)


## Evluate Ml model

score() : In multilabel classification, this function computes subset accuracy: the set of labels predicted for a sample must exactly match the corresponding set of labels in y_true.



In [ ]:
print ("MNB score for train",clf_mnb.score(X_train, y_train))
print ("MNB score for test",clf_mnb.score(X_test, y_test))
print ("SVM score for train",clf_svm.score(X_train, y_train))
print ("SVM score for test",clf_svm.score(X_test, y_test))

tried naive bayes, random forest and svm and select the two highest accuracy

# Deep Learning Model

## preprocessing for DL

### tokenizer

converts Python text strings to streams of token objects, where each token object is a separate word, punctuation sign, number/amount, date, e-mail, URL/URI, etc.

In [ ]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df.text.values)
X = tokenizer.texts_to_sequences(df.text.values)


### padding

padding each vector to fit the dl model required format

takes in a sequence of data-points gathered at equal intervals, along with time series parameters such as stride, length of history, etc., to produce batches for training/validation.

In [ ]:
X = pad_sequences(X)

 ### split data into train and test

encoded the labels of the three classes to fit the model required format

In [ ]:
Y = pd.get_dummies(df['airline_sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42, stratify=Y)
X_train.shape

 ### split train data to train and validation set

The validation dataset provides an unbiased evaluation of a model fit on the training dataset while tuning the model's hyperparameters

In [ ]:
X_vald = X_train[:500]
Y_vald = Y_train[:500]
x_train = X_train[500:]
y_train = Y_train[500:]

## Create DL Model

 I used: Tensorflow and Keras to define the neural network
 
 Embedding: will learn an embedding for all of the words in the training datase
 
 Dropout: A Simple Way to Prevent Neural Networks from Overfitting
 
 
 LSTM :  allows you to specify the merge mode, that is how the forward and backward outputs should be combined before being passed on to the next layer
 
 
 Dense: recieves input from all the neurons in the previous layer, thus densely connected
 
 
 categorical_crossentropy: as our target to classify three labels

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

## train the model

The batch size defines the number of samples that will be propagated through the network.

In [ ]:
batch_size = 512
history = model.fit(x_train, 
                    y_train, 
                    epochs = 10, 
                    batch_size=batch_size, 
                    validation_data=(X_vald, Y_vald))

## visualize the results

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = history.history['acc']
val_acc = history.history['val_acc']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

## Evaluate the model

Returns the loss value & metrics values for the model in test mode.

In [ ]:
scores = model.evaluate(X_test, Y_test)
scores